In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation,Lambda
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from sklearn.model_selection import train_test_split
from keras.models import model_from_json
from keras import optimizers
from keras.optimizers import Adadelta
import keras.backend as K
import string
import csv
np.random.seed(1)

Using TensorFlow backend.


# Loading & Preprocessing

In [3]:
data = pd.read_csv('/content/drive/My Drive/txt_datsets/quora/train.csv')
print(data.shape)

(363861, 6)


In [37]:
sub_data = data[['question1','question2','is_duplicate']] 
print(sub_data.shape)

(363861, 3)


In [38]:
sub_data.dropna( inplace=True)
print(sub_data.shape)

(363858, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
sub_data.duplicated(subset={'question1','question2','is_duplicate'},).value_counts()
sub_data.drop_duplicates(subset={'question1','question2','is_duplicate'},keep="first",inplace=True)
print(sub_data.shape)

(363858, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [40]:
print(sub_data.head())

                                           question1  ... is_duplicate
0  What is the step by step guide to invest in sh...  ...            0
1  What is the story of Kohinoor (Koh-i-Noor) Dia...  ...            0
2  How can I increase the speed of my internet co...  ...            0
3  Why am I mentally very lonely? How can I solve...  ...            0
4  Which one dissolve in water quikly sugar, salt...  ...            0

[5 rows x 3 columns]


In [41]:
pos = sub_data[sub_data["is_duplicate"]>=0.5]
print(pos.shape)

(135175, 3)


In [42]:
neg = sub_data[sub_data["is_duplicate"]<0.5]
print(neg.shape)

(228683, 3)


In [43]:
neg = neg.sample(n=pos.shape[0],replace=True)
print(neg.shape)

(135175, 3)


In [44]:
sub_data = pd.concat([pos, neg])
print(sub_data.shape)

(270350, 3)


In [45]:
print(sub_data[sub_data['is_duplicate']==1].shape[0])
print(sub_data[sub_data['is_duplicate']==0].shape[0])

135175
135175


# Splitting into training and validation data sets

In [46]:
X_train, X_test, y_train, y_test = train_test_split(np.array(sub_data[['question1','question2']]),
                                                    np.array(sub_data[['is_duplicate']]), test_size=0.2, 
                                                    random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(216280, 2) (216280, 1)
(54070, 2) (54070, 1)


In [47]:
X1_train = X_train[:,0]
X2_train = X_train[:,1]

X1_test = X_test[:,0]
X2_test = X_test[:,1]

print(X1_train.shape, X2_train.shape)
print(X1_test.shape, X2_test.shape)

(216280,) (216280,)
(54070,) (54070,)


# Loading glove vectors

In [0]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [0]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('/content/drive/My Drive/toxicity/glove.6B.300d.txt')

In [48]:
max_len = 0
avg_len = 0
num = 0
for i in sub_data.question1:
  try:
    tmp = len(i.translate(str.maketrans('','',string.punctuation)).lower().split())
    num+=1
    avg_len += tmp
    if tmp>max_len:
      max_len = tmp
  except:
    print(num,i)

for i in sub_data.question2:
  try:
    tmp = len(i.translate(str.maketrans('','',string.punctuation)).lower().split())
    num+=1
    avg_len += tmp
    if tmp>max_len:
      max_len = tmp
  except:
    print(num,i)

avg_len = avg_len/num
print(max_len)
print(avg_len)

237
10.789657850933974


# Converting words to word indices

In [0]:
def sentences_to_indices(X, word_to_index, seq_len):
    m = X.shape[0]                                
    X_indices = np.zeros((m, seq_len))
    
    for i in range(m): 
        tmp = X[i].translate(str.maketrans('','',string.punctuation)).lower().split()
        j = 0
        start_index =seq_len-len(tmp)
        if start_index<0:
          start_index =0
        for w in tmp:
          if j>= seq_len:
            break
          try:
            X_indices[i, start_index+j] = word_to_index[w]
            j = j+1
          except KeyError:
            pass
            
    
    return X_indices

In [50]:
X1_train_indices = sentences_to_indices(X1_train,word_to_index, 25)
X2_train_indices = sentences_to_indices(X2_train,word_to_index, 25)
X1_test_indices = sentences_to_indices(X1_test,word_to_index, 25)
X2_test_indices = sentences_to_indices(X2_test,word_to_index, 25)
print(X1_train_indices.shape)
print(X2_train_indices.shape)
print(X1_test_indices.shape)
print(X2_test_indices.shape)

(216280, 25)
(216280, 25)
(54070, 25)
(54070, 25)


# Model 

In [0]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1            
    emb_dim = word_to_vec_map["cucumber"].shape[0]  
    emb_matrix = np.zeros((vocab_len, emb_dim))
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [52]:
n_hidden = 64
batch_size = 256
n_epoch = 25

def L1_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

input1 = Input(shape=(None,))
input2 = Input(shape=(None,))

embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)

encoded1 = embedding_layer(input1)
encoded2 = embedding_layer(input2)

shared_lstm = LSTM(n_hidden)

output1 = shared_lstm(encoded1)
output2 = shared_lstm(encoded2)

l1_distance = Lambda(function=lambda x: L1_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([output1, output2])

l1_lstm = Model([input1, input2], [l1_distance])




In [53]:
l1_lstm.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    120000300   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 64)           93440       embedding_1[0][0]          

In [0]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# Training the model

In [55]:
sgd = optimizers.SGD(lr=0.1, decay=1e-4, momentum=0.9, nesterov=True)
l1_lstm.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy',f1_m])

l1_lstm_trained = l1_lstm.fit([X1_train_indices, X2_train_indices], y_train, batch_size=batch_size, nb_epoch=n_epoch,
                            validation_data=([X1_test_indices, X2_test_indices], y_test))#,class_weight={0:0.3,1:0.7}




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  




Train on 216280 samples, validate on 54070 samples
Epoch 1/25
216280/216280 [==============================] - 64s 297us/step - loss: 0.6306 - acc: 0.7114 - f1_m: 0.6797 - val_loss: 0.5711 - val_acc: 0.7484 - val_f1_m: 0.7454
Epoch 2/25
216280/216280 [==============================] - 62s 289us/step - loss: 0.5574 - acc: 0.7580 - f1_m: 0.7500 - val_loss: 0.5478 - val_acc: 0.7663 - val_f1_m: 0.7611
Epoch 3/25
216280/216280 [==============================] - 62s 286us/step - loss: 0.5384 - acc: 0.7715 - f1_m: 0.7662 - val_loss: 0.5372 - val_acc: 0.7742 - val_f1_m: 0.7734
Epoch 4/25
216280/216280 [==============================] - 62s 287us/step - loss: 0.5262 - acc: 0.7808 - f1_m: 0.7768 - val_loss: 0.5294 - val_acc: 0.7747 - val_f1_m: 0.7622
Epoch 5/25
216280/216280 [==============================] - 62s 285us/step - loss: 0.5169 - acc: 0.7874 - f1_m: 0.7842 - val_loss: 0.5220 - val_acc: 0.7826 - val_f1_m: 0.7852
Epoch 6/25
216280/216280 [==============================] - 62s 284us/st

# Saving the model after removing last layer

In [0]:
model = Model([input1, input2],[output1,output2])

In [57]:
model_json = model.to_json()
with open("/content/drive/My Drive/codes/quora/lstm_25e.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("/content/drive/My Drive/codes/quora/lstm_25e.h5")
print("Saved model to disk")



Saved model to disk
